In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/sentence-simplification/valid.csv
/kaggle/input/sentence-simplification/train.csv
/kaggle/input/sentence-simplification/test.csv


In [ ]:
! pip install -q -U bitsandbytes
! pip install -q -U git+https://github.com/huggingface/transformers.git
! pip install -q -U git+https://github.com/huggingface/peft.git
#pip install -q -U git+https://github.com/huggingface/accelerate.git
#current version of Accelerate on GitHub breaks QLoRa
#Using standard pip instead
! pip install -q -U accelerate
! pip install -q -U datasets
! pip install evaluate
! pip install rouge_score

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cuml 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
dask-cudf 23.8.0 requires cupy-cuda11x>=12.0.0, which is not installed.
cudf 23.8.0 requires pandas<1.6.0dev0,>=1.3, but you have pandas 2.0.3 which is incompatible.
cudf 23.8.0 requires protobuf<5,>=4.21, but you have protobuf 3.20.3 which is incompatible.
cuml 23.8.0 requires dask==2023.7.1, but you have dask 2023.12.0 which is incompatible.
cuml 23.8.0 requires distributed==2023.7.1, but you have distributed 2023.12.0 which is incompatible.
dask-cuda 23.8.0 requires dask==2023.7.1, but you have dask 2023.12.0 which is incompatible.
dask-cuda 23.8.0 requires distributed==2023.7.1, but you have distributed 2023.12.0 which is incompatible.
dask-cuda 23.8.0 requires pandas<1.6.0dev0,

In [ ]:
df = pd.read_csv("/kaggle/input/sentence-simplification/train.csv")
df

,Unnamed: 0,0,1
0,0,Lakenheath is host to the largest USAF base in...,Lakenheath is a village and civil parish in Su...
1,1,Danbury is a city in northern Fairfield County...,Danbury is a city of Connecticut in the United...
2,2,"In 1997 , after a failed engagement to a Musli...","In 1997 , after a marriage engagement to a Mus..."
3,3,"It initially was completely dark , but as Voya...","It originally was completely dark , but as Voy..."
4,4,The Thinker was originally meant to depict Dan...,The Thinker was originally meant to depict Dan...
...,...,...,...
138408,138408,He is best known for his roles as David Healy ...,Galecki is best known for playing the role of ...
138409,138409,"Today NRC is organised as an independent , pri...","Today , the NRC is an independent , private or..."
138410,138410,He is probably best known internationally for ...,He is probably best known for playing Iolaus i...
138411,138411,De Lesseps then retired from the diplomatic se...,De Lesseps then retired from the diplomatic se...


In [ ]:
df['0']

0         Lakenheath is host to the largest USAF base in...
1         Danbury is a city in northern Fairfield County...
2         In 1997 , after a failed engagement to a Musli...
3         It initially was completely dark , but as Voya...
4         The Thinker was originally meant to depict Dan...
                                ...                        
138408    He is best known for his roles as David Healy ...
138409    Today NRC is organised as an independent , pri...
138410    He is probably best known internationally for ...
138411    De Lesseps then retired from the diplomatic se...
138412    Despite the number , this is a north -- south ...
Name: 0, Length: 138413, dtype: object

In [ ]:
from transformers import BartTokenizer, BartForConditionalGeneration, AutoModelForSeq2SeqLM, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# Load pre-trained BART model and tokenizer
model_name = "facebook/bart-large-cnn" # Flan-T5-Large
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

In [ ]:
from datasets import load_dataset
train = load_dataset("csv", data_files="/kaggle/input/sentence-simplification/train.csv")
test = load_dataset("csv", data_files="/kaggle/input/sentence-simplification/test.csv")
val = load_dataset("csv", data_files="/kaggle/input/sentence-simplification/valid.csv")

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):
/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

/opt/conda/lib/python3.10/site-packages/pyarrow/pandas_compat.py:373: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if _pandas_api.is_sparse(col):


In [ ]:
def tokenize_function(example):
    example['input_ids'] = tokenizer(example["0"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    example['labels'] = tokenizer(example["1"], padding="max_length", truncation=True, return_tensors="pt").input_ids
    return example

# The dataset actually contains 3 diff splits: train, validation, test.
# The tokenize_function code is handling all data across all splits in batches.
train_data = train.map(tokenize_function, batched=True)
test_data = test.map(tokenize_function, batched=True)
val_data = val.map(tokenize_function, batched=True)

Map:   0%|          | 0/138413 [00:00<?, ? examples/s]

Map:   0%|          | 0/359 [00:00<?, ? examples/s]

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [ ]:
from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq,
)

training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=1,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=1,  # demo
    per_device_eval_batch_size=1,
    # learning_rate=3e-05,
    warmup_steps=10,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=50,
    max_steps=68,
    save_total_limit=3,
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data['train'],
    eval_dataset=val_data['train'],
    tokenizer=tokenizer,
#     compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
50,5.413100


TrainOutput(global_step=68, training_loss=4.488602301653693, metrics={'train_runtime': 152.8118, 'train_samples_per_second': 0.89, 'train_steps_per_second': 0.445, 'total_flos': 294726225887232.0, 'train_loss': 4.488602301653693, 'epoch': 0.0})

In [ ]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples,
        padding="max_length",
        truncation=True,
#         max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


# model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# test_samples = validation_data_txt.select(range(16))

# summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]

In [ ]:
test1 = pd.read_csv("/kaggle/input/sentence-simplification/test.csv")
test1

,Unnamed: 0,0,1
0,0,One side of the armed conflicts is composed ma...,One side of the armed conflicts is made of Sud...
1,1,"Jeddah is the principal gateway to Mecca , Isl...","Jeddah is the main gateway to Mecca, Islam's h..."
2,2,The Great Dark Spot is thought to represent a ...,The Great Dark Spot is thought to represent a ...
3,3,"His next work , Saturday , follows an especial...",His next work at Saturday will be a successful...
4,4,"The tarantula , the trickster character , spun...","The tarantuala, the trickster, spun a black ch..."
...,...,...,...
354,354,Although the name suggests that they are locat...,Eventhough the name evokes that they are place...
355,355,"There he had one daughter , later baptized as ...",He had one daughter named Mary Ann Fisher Powe...
356,356,"During an interview , Edward Gorey mentioned t...","During an interview, Edward Gorey said that Ba..."
357,357,The string can vibrate in different modes just...,The string can vibrate in different modes just...


In [ ]:
l = []
for i in range(0,len(test1)) :
    l.append(generate_summary(test1.iloc[i,-1], model)[1])
l

[['\xa0 the and\xa0 in a far Jan,.) ;-peunningsp,"inentaint'],
 [' the\xa0 a inJanga," aimediga andance part theJ\xa0 the\xa0\xa0\xa0," the\xa0 the'],
 ['\xa0 a the theazingpe in and The-olargal,"The Aanga a\xa0\xa0 the the\xa0 the a the the\xa0\xa0'],
 ['\xa0 a the inroad andpe-iding --g fitaineorter " social partinentanceunless, or,"rel \''],
 ['\xa0 and in asp\xa0 the,"orterThe,-esteranceazing castre Araburtiff the\xa0\xa0esterunningrelaintanga part Aidingondonply from'],
 ['\xa0 in He the.) effectively," stretch aondon and,pe Ataboola descend Theg far'],
 [' theondon a\xa0 in and far hasroad.),"orterThe Thepeunless,git'],
 [' the.) a\xa0 in and,\xa0 the the\xa0 the the in the the\xa0- The from farThe'],
 ['\xa0 the a.) Saintwise football and in far-orter ofaupeTheancestondonunningult'],
 [' in the\xa0 has a and- --ondoniff TheiveitcolThepeorter,'],
 ['\xa0\xa0 a the in and.)ondon A a a a\xa0 a the\xa0\xa0\xa0 a a\xa0 the a- football'],
 ['it\xa0 andItcr a theunning in believedunles

In [ ]:
import evaluate
predictions = test1.iloc[:,2]
references = l
bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)
print(results)

{'bleu': 0.0, 'precisions': [0.14948920212078107, 0.0027122321670735015, 0.0, 0.0], 'brevity_penalty': 1.0, 'length_ratio': 1.4492128935532234, 'translation_length': 7733, 'reference_length': 5336}


In [ ]:
l1 = [i[0] for i in l]

In [ ]:
predictions = test1.iloc[:,2]
rouge = evaluate.load('rouge')
references = l1
results = rouge.compute(predictions=predictions,references=references)
print(results)

{'rouge1': 0.14658405783426487, 'rouge2': 0.0034387322186428242, 'rougeL': 0.12325636393739148, 'rougeLsum': 0.12299185791572684}


In [ ]:
! pip install sacrebleu
! pip install sacremoses

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for sacrebleu from https://files.pythonhosted.org/packages/0a/a6/2ac47e71e526bbcd97ea08f20d9ef7d3852e2594ec7b2d55f5d2bbfd7aae/sacrebleu-2.3.3-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.5 MB/s eta 0:00:00
  Obtaining dependency information for portalocker from https://files.pythonhosted.org/packages/17/9e/87671efcca80ba6203811540ed1f9c0462c1609d2281d7b7f53cef05da3d/portalocker-2.8.2-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.4/106.4 kB 9.7 MB/s eta 0:00:00


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Obtaining dependency information for sacremoses from https://files.pythonhosted.org/packages/0b/f0/89ee2bc9da434bd78464f288fdb346bc2932f2ee80a90b2a4bbbac262c74/sacremoses-0.1.1-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 18.7 MB/s eta 0:00:00a 0:00:01


In [ ]:
from evaluate import load
sari = evaluate.load("sari")
sources= test1.iloc[:,1]
predictions= l1
references= [[i] for i in test1.iloc[:,2] ]
sari_score = sari.compute(sources=sources, predictions=predictions, references=references)

In [ ]:
sari_score

{'sari': 15.645538983428134}